# Basic setups

In [8]:
# 수 분 걸릴 수 있습니다

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_package_stable/')
# sys.path.append('/home/users/pjh/scripts/python_genome_package_dev/')
import os

import pysam

from handygenome import common
from handygenome.variant.vcfspec import Vcfspec
from handygenome.variant.variantplus import VariantPlus
from handygenome.igvhandle import IGVHandle
from handygenome.align import realign

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
FASTA_HG19 = common.DEFAULT_FASTAS['hg19']
FASTA_HG38 = common.DEFAULT_FASTAS['hg38']
CHROMDICT_HG19 = common.ChromDict(fasta=FASTA_HG19)
CHROMDICT_HG38 = common.ChromDict(fasta=FASTA_HG38)

In [3]:
# jupyter를 실행한 node와 같은 node에서 IGV를 띄워놔야 합니다
# "port" argument에는 IGV에서 설정한 port와 동일한 것을 적어야 합니다

igv = IGVHandle(port=60387)

# Main

### parameter 세팅

In [21]:
# "bam_paths" 딕셔너리에 bam file들의 id 및 절대경로를 key-value로 적어주면 됩니다. id는 임의로 적어도 됩니다.
# bam_paths = {}
# 예시
DB3_topdir = '/home/users/team_projects/Lineage_tracing/DB3/02_BAM'
bam_paths = {
    '3_LLL-1_Fb_001C3': os.path.join(DB3_topdir, '3_LLL-1_Fb_001C3.s.md.ir.br.cram'),
    '3_LSM_001B10': os.path.join(DB3_topdir, '3_LSM_001B10.s.md.ir.br.cram'),
}


# realign 할 위치를 적어줍니다. variant가 있는 곳 근처의 아무런 범위로나 하면 됩니다.
# locus = ('17', 3_468_331, 3_468_331)
locus = ('18', 3_262_072, 3_262_072)  
    # CHROM, start, end 순으로 적어주면 됩니다. 
    # CHROM은 str type으로 해야 합니다.
    # start, end는 1-based inclusive 좌표계로 적어야 합니다.    

    
# realigned bam이 저장될 디렉토리를 지정해 줘야 합니다. 새 디렉토리를 하나 만든 다음에 절대경로를 assign 하세요.
# realigned_bams_topdir = ''    
# 예시: 
realigned_bams_topdir = '/home/users/pjh/practice/realign'    

#### 이 아래부터는 모두 그냥 실행하면 됩니다.

In [22]:
bam_dict = {key: pysam.AlignmentFile(val) for key, val in bam_paths.items()}
chrom = locus[0]
start0 = locus[1] - 1
end0 = locus[2]
loci = [locus]

### Loading original BAMs

In [23]:
igv.cmd('new')
igv.load(list(bam_paths.values()))
igv.goto(loci, width=200)  # width 값을 조정하면 더 넓은 범위를 볼 수 있습니다.
igv.cmd('collapse')  # expand 하고싶으면 collapse 대신에 expand 쓰면 됩니다.

OK
OK
OK
OK
OK


### Doing realignment - 조건에 따라 수십초 걸리기도 합니다 (개선할 예정). 1분 넘게 안 끝나면 무한루프 등의 에러일 가능성이 높습니다 ㅠㅠ

In [24]:
msrpileup_ser = realign.MultisampleRealignerPileupSeries(
    FASTA_HG19, bam_dict, chrom, start0, end0,
    inactive_padding=10,  # variation이 있는 영역 주변으로 variation 없는 여유 공간을 얼마나 확보할 것인가를 나타냄
    verbose=False,  # verbose를 True로 하면 debugging message 출력
)

### Writing realigned bam files

In [25]:
import re

realigned_bam_paths = {
    key: os.path.join(realigned_bams_topdir, re.sub(r'(bam|cram)$', r'realigned.bam', os.path.basename(val)))
    for key, val in bam_paths.items()
}

msrpileup_ser.write_realigned_reads(realigned_bam_paths, padding=500)
realigned_bams = {sampleid: pysam.AlignmentFile(bam_path) for sampleid, bam_path in realigned_bam_paths.items()}

### Showing on IGV

In [26]:
vcfspec = msrpileup_ser.get_result_vcfspecs()[0]
vp = VariantPlus.from_vcfspec(vcfspec)

for key, val in realigned_bams.items():
    vp.show_igv(igv, bam=val, tmpbam_name=f'{key}_realigned.bam', viewaspairs=False)

igv.cmd('sort POSITION')
igv.cmd('collapse')

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


### VCF entry - 위치가 인접한 것들을 합쳐서 multi-ALT entry로 출력

In [ ]:
msrpileup_ser.get_result_vcfspecs()

### Read annotation - supporting read count 출력

In [29]:
for sampleid, bam in realigned_bams.items():
    vp.set_readstats(sampleid, bam)

In [30]:
for sampleid, db in vp.readstats_dict.items():
    print(sampleid, db['rppcounts'])

3_LLL-1_Fb_001C3 {None: 5, -1: 4, 0: 16, 1: 17, 'softclip_overlap': 0}
3_LSM_001B10 {None: 5, -1: 3, 0: 9, 1: 14, 'softclip_overlap': 0}
